## Modules

In [1]:
import os
import sys
sys.path.append('..')


import plotly.graph_objects as go
from plotly.subplots import make_subplots

from bsoption.bsmodel import BSModel
from bsoption.nyopchain import NYopchain
Opchain = NYopchain()

from litedata import *
from litedata.loadyahoodata import getyahoodata

### US stock list setting

In [ ]:
from configparser import ConfigParser
configchain = ConfigParser()
configchain.read('setting_yahoochain.ini')
chainpath = configchain['paths'].get('chainpath')

configassets = configchain['assets']
etflist = list(configassets.get('etf').split(','))
chipslist = list(configassets.get('chip').split(','))
assetlist = etflist + chipslist

## Gamma Exposure (GEX)

### OHLC of all stocks

In [ ]:
dfohlcall = getyahoodata(assetlist)

### Option chain with closing price >= 0.05 with greeks

In [ ]:
def loadopchain(daystr='2022-10-03', expiry='2022-10-14', asset='NVDA', bounds=(0, 1000), opbound=0.05):
    """Obtain all options of the same expiry with closing price > 0.01 with greeks."""
    # inputdict for `.loadopdata()` method and query
    inputdict = {'asset': asset,  'optype': ('C', 'P'), 
                 'strike_lowerbound': bounds[0], 'strike_upperbound': bounds[1],  
                 'startexpiry': expiry,  'endexpiry': expiry,  'starttd': daystr,  'endtd': daystr}
    dfop = Opchain.loadopdata(inputdict)
    # Filter strike price with BOTH call & put price over 1.00
    dfcall = dfop[(dfop['optype'] == 'C') & (dfop['mid'] >= opbound)]
    dfput = dfop[(dfop['optype'] == 'P') & (dfop['mid'] >= opbound)]
    strikeset = set(dfcall['strike']).intersection(set(dfput['strike']))
    dfop = dfop[dfop['strike'].isin(strikeset)]
    # Underlying close price
    spotprice = dfohlcall.loc[daystr, f'{asset}_cl']
    dfop[f'{asset}_cl'] = spotprice
    # Dummy column of `BSmodel()` object
    tdays = (datetime.strptime(expiry, '%Y-%m-%d') - datetime.strptime(daystr, '%Y-%m-%d')).days
    dfop['BS'] = dfop.apply(lambda row: BSModel(spotprice, row['strike'], tdays, row['iv'] / 100), axis=1)
    # split option dataframe into call & put and compute delta and theta separately
    dfcall = dfop[dfop['optype'] == 'C']
    dfput = dfop[dfop['optype'] == 'P']
    dfcall['delta'] = dfcall['BS'].apply(lambda x: x.cdelta)
    dfcall['theta'] = dfcall['BS'].apply(lambda x: x.ctheta)
    dfput['delta'] = dfput['BS'].apply(lambda x: x.pdelta)
    dfput['theta'] = dfput['BS'].apply(lambda x: x.ptheta)
    dfop = pd.concat([dfcall, dfput], axis=0)
    # Compute vega & gamma columns (irrelevant of call/put)
    dfop['vega'] = dfop['BS'].apply(lambda x: x.vega)
    dfop['gamma'] = dfop['BS'].apply(lambda x: x.gamma)
    # Drop dummy column
    dfop.drop('BS', axis=1, inplace=True)
    
    return dfop

In [ ]:
asset0 = 'NVDA'
dfop1, info1 = Opchain.loaddayopchain(dfohlcall, tradeday='2022-08-26', expiry='2022-09-16', asset=asset0)
dfop1

### Gamma exposure function

In [ ]:
def getgex(dfchain, asset='NVDA'):
    """Obtain gamma exposure of each option contract per 1% move of underlying."""
    ## Attach GEX column
    dfgex = dfchain.copy()
    putfactor = lambda x: -1 if (x == 'P') else 1
    colfactor = pd.to_numeric(((dfgex['optype']).apply(putfactor)))
    dfgex['GEX'] = dfgex['gamma'] * dfgex['oi'] * colfactor * 100
    # Split into call & put dataframe
    collist = ['tradedate', 'expiry', f'{asset}_cl', 'strike', 'mid', 'iv', 'oi', 'delta', 'gamma', 'GEX']
    dfcall = dfgex[dfgex['optype'] == 'C'][collist]
    dfcall.rename(columns={field: f'c_{field}' for field in collist[4:]}, inplace=True)
    dfcall['c_GEX'] *= dfgex[f'{asset}_cl'].iloc[0] / 100
    dfcall.set_index('strike', inplace=True)
    dfput = dfgex[dfgex['optype'] == 'P'][collist[3:]]
    dfput.rename(columns={field: f'p_{field}' for field in collist[4:]}, inplace=True)
    dfput['p_GEX'] *= dfgex[f'{asset}_cl'].iloc[0] / 100
    dfput.set_index('strike', inplace=True)
    # Concat along columns
    dfgex = pd.concat([dfcall, dfput], axis=1)
    # Net GEX (call GEX + put GEX)
    dfgex['GEX'] = dfgex['c_GEX'] + dfgex['p_GEX']
    for col in ['c_GEX', 'p_GEX', 'GEX']:
        dfgex[col] = np.round(dfgex[col], 4)
    
    return dfgex

In [ ]:
dfgex1 = getgex(dfop1, asset=asset0)
dfgex1[(dfgex1['c_mid'] > 1) & (dfgex1['p_mid'] > 1)]

### Visualize GEX

In [ ]:
def gettotalgex(dfgex):
    """Obtain call-GEX, put-GEX and total-GEX."""
    return round(dfgex['c_GEX'].sum(), 2), round(dfgex['p_GEX'].sum(), 2), round(dfgex['GEX'].sum(), 2)

def plotgex(dfgex, asset='NVDA'):
    """Plot call GEX, put GEX and net GEX."""
    tdate = (dfgex['tradedate'].iloc[0]).strftime('%Y-%m-%d')
    expiry = (dfgex['expiry'].iloc[0]).strftime('%Y-%m-%d')
    spotprice = dfgex[f'{asset}_cl'].iloc[0]
    gexsum_c, gexsum_p, gexsum = gettotalgex(dfgex)
    
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.125, row_heights=[8.0, 8.0],
                        specs=[[{"type": "bar"}], [{"type": "bar"}]], 
                        subplot_titles=(f'Call-GEX: {gexsum_c}, Put-GEX: {gexsum_p}', f'Net GEX: {gexsum}'))
        
    fig.add_trace(go.Bar(x=dfgex.index, y=dfgex['c_GEX'], name="Call GEX",
                         marker=dict(color='#34dd74', line=dict(color='#34dd74', width=2))), 
                  row=1, col=1)
    
    fig.add_trace(go.Bar(x=dfgex.index, y=dfgex['p_GEX'], name="Put GEX",
                         marker=dict(color='#dd3462', line=dict(color='#dd3462', width=2))), 
                  row=1, col=1)
    
    fig.add_trace(go.Bar(x=dfgex.index, y=dfgex['GEX'], name="GEX",
                         marker=dict(color='#1956ee', line=dict(color='#1956ee', width=2))), 
                  row=2, col=1)
    
    titletext = f'{asset} GEX of expiry {expiry} on {tdate} at {spotprice} \n'
    
    fig.update_layout(title=titletext, title_x=0.5, width=1000, height=800)
    fig.show()

In [ ]:
plotgex(dfgex1, asset=asset0)

### Compute GEX for general spot price 

In [ ]:
def getnewgex(dfgex, asset='NVDA', spotprice=180):
    """Obtain total GEX of an option chain with arbitary underlying price."""
    # Alter underlying price
    dfnewgex = dfgex.copy()
    dfnewgex[f'{asset}_cl'] = spotprice
    # Recompute option price, delta and gamma
    dfnewgex.reset_index(inplace=True)
    tdate = dfnewgex['tradedate'].iloc[0]
    expiry = dfnewgex['expiry'].iloc[0]
    tdays = (expiry - tdate).days
    colbs_c = dfnewgex.apply(lambda row: BSModel(spotprice, row['strike'], tdays, row['c_iv'] / 100), axis=1)
    dfnewgex['c_close'] = np.round(colbs_c.apply(lambda x: x.cprice), 2)
    dfnewgex['c_delta'] = np.round(colbs_c.apply(lambda x: x.cdelta), 4)
    dfnewgex['c_gamma'] = colbs_c.apply(lambda x: x.gamma)
    colbs_p = dfnewgex.apply(lambda row: BSModel(spotprice, row['strike'], tdays, row['p_iv'] / 100), axis=1)
    dfnewgex['p_close'] = np.round(colbs_p.apply(lambda x: x.pprice), 2)
    dfnewgex['p_delta'] = np.round(colbs_p.apply(lambda x: x.pdelta), 4)
    dfnewgex['p_gamma'] = colbs_p.apply(lambda x: x.gamma)    
    # GEX
    dfnewgex['c_GEX'] = np.round(dfnewgex['c_gamma'] * dfnewgex['c_oi'] * spotprice, 4) 
    dfnewgex['p_GEX'] = np.round(dfnewgex['p_gamma'] * dfnewgex['p_oi'] * -1 * spotprice, 4)
    dfnewgex['GEX'] = np.round(dfnewgex['c_GEX'] + dfnewgex['p_GEX'], 4)
    # Set back `strike` as index
    dfnewgex.set_index('strike', inplace=True)
    
    return dfnewgex

### Zero Gamma Level

In [ ]:
def get0gamma(dfchain, asset='NVDA', spotbound=(160, 200), interval=2.5):
    """Obtain zero gamma level."""
    # GEX according to true underlying price
    dfgex = getgex(dfchain, asset)
    spotprice = dfgex[f'{asset}_cl'].mean()
    cgex0, pgex0, ngex0 = gettotalgex(dfgex)
    # Obtain GEX for all spot price levels
    spotlevels = np.arange(spotbound[0], spotbound[1], interval)
    dfsumgex = pd.DataFrame()
    for level in spotlevels:
        dfnewgex = getnewgex(dfgex, asset, level)
        cgex, pgex, ngex = gettotalgex(dfnewgex)
        dfsumgex.loc[level, 'c_GEX'] = cgex
        dfsumgex.loc[level, 'p_GEX'] = pgex
        dfsumgex.loc[level, 'GEX'] = ngex
    # Obtain zero gamma level by linear interpolation
    if (dfsumgex['GEX'].min() * dfsumgex['GEX'].max()) > 0:
        zerogexlevel = None
    else:
        for p1, p2 in zip(dfsumgex.index[:-1], dfsumgex.index[1:]):
            level1 = dfsumgex.loc[p1, 'GEX']
            level2 = dfsumgex.loc[p2, 'GEX']
            if level1 * level2 < 0:
                zerogexlevel = round((p1 * level2 - p2 * level1) / (level2 - level1), 2)
                break
    # Visualize GEX at different spot price 
    fig = make_subplots(rows=1, cols=1, shared_xaxes=True, row_heights=[6.0], specs=[[{"type": "scatter"}]])
    
    fig.add_trace(go.Scatter(x=dfsumgex.index, y=dfsumgex['GEX'], mode='lines+markers', name='spot levels'), 
                  row=1, col=1)
    
    fig.add_trace(go.Scatter(x=[spotprice], y=[ngex0], mode='markers', name='spot price', 
                             marker=dict(size=20, color='#66dc19')), row=1, col=1)
    
    if zerogexlevel != None:
        fig.add_trace(go.Scatter(x=[zerogexlevel], y=[0], mode='markers', name='0-gamma level', 
                                 marker=dict(size=20, color=' #ee195a')), row=1, col=1)
        
    tdate = dfchain['tradedate'].iloc[0].strftime('%Y-%m-%d')
    expiry = (dfchain['expiry'].iloc[0]).strftime('%Y-%m-%d')
    fig.update_layout(title=f'Gamma levels of {asset} of expiry {expiry} on {tdate}', 
                      title_x=0.5, width=1000, height=800)
    fig.show()

    return dfsumgex, zerogexlevel

In [ ]:
lowb1 = dfop1[f'{asset0}_cl'].mean() * 0.8
uppb1 = dfop1[f'{asset0}_cl'].mean() * 1.2
dfsumgex1, zerogex1 = get0gamma(dfop1, asset=asset0, spotbound=(lowb1, uppb1))